# prep

In [87]:
from deap import base
from deap import creator
from deap import tools

import pandas as pd
import random
import numpy as np
from sympy import *
from graphviz import *

import matplotlib.pyplot as plt
import seaborn as sns
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 

def pow_3(x1):
    f = x1**3
    return f
pow_3 = make_function(function=pow_3,name='pow3',arity=1)

converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y,
    'sin': lambda x    : sin(x),
    'cos': lambda x    : cos(x),
    'log': lambda x    : ln(x),
    'inv': lambda x: 1/x,
    'sqrt': lambda x: x**0.5,
    'pow3': lambda x: x**3
}


In [ ]:
nsample = 400
sig = 0.2
x = np.linspace(-50, 50, nsample)
X = np.column_stack((x/5, 10*np.sin(x), (x-5)**3, np.ones(nsample)))
beta = [0.01, 1, 0.001, 5.]
y_true = np.dot(X, beta)
y = y_true + sig * np.random.normal(size=nsample)
df = pd.DataFrame()
df['x']=x
df['y']=y

y = df['y']
X = df[['x']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# , pow_3
# First Test
function_set = ['add', 'sub', 'mul', 'div', 'cos', 'sin', 'neg', 'inv', 'log']

est_gp = SymbolicRegressor(
    population_size=5000
    ,metric="rmse" # mean absolute error
    ,function_set=function_set
    ,generations=100
    ,stopping_criteria=0.01
    ,p_crossover=0.7
    ,p_subtree_mutation=0.1
    ,p_hoist_mutation=0.05
    ,p_point_mutation=0.1
    ,max_samples=0.9
    ,verbose=1
    ,parsimony_coefficient=0.001
    ,random_state=0
    ,feature_names=X_train.columns
)
                          
est_gp.fit(X_train, y_train)
print('R2:',est_gp.score(X_test,y_test))
next_e = sympify((est_gp._program), locals=converter)
next_e   

print(est_gp._program) 
df['prediction'] = est_gp.predict(X)
df.to_csv('c:/data/test.csv', index=False)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    14.13      8.67143e+06       16          22.8598          25.0129      4.15m
   1    12.89           143.17       54          20.2202          21.9061      3.99m
   2    16.32          121.615       42          18.9818          17.0851      4.58m
   3    22.56           130.12       58          15.2266          15.9139      4.97m
   4    33.26          551.887       75          14.9481          12.9553      5.58m
   5    42.61          1266.45       71          14.3451          18.0289      6.52m
   6    42.75          945.041       60          14.0232          12.1891      6.26m
   7    50.06          700.824      143          13.2886          9.83953      6.76m
   8    67.01          912.532       62          11.1667          8.59271  